# K-Means Clustering

In this notebook, we'll take a look at the most common clustering algorithm:  *K-Means Clustering*.  Let's look at the name:

* `K` - the algorithm assumes that the data can be split up into $K$ *distinct* sets.  In other words, every item in the data set is assigned to one - and only one - cluster.  This is known as a *partitioning*.
* `Means` - the algorithm attempts to define a set of *centroids* - the *representatives* of each cluster.  Centroids are determined by taking the *mean* of all members of this cluster.  By using the mean, the algorithm tries to reduce the squared error between the centroid and all members of the cluster.

The algorithm is composed of two parts:
1.  Assignment - Assign each data point to the cluster of the closest centroid ('mean').  Initial centroids are selected using various techniques - let's assume for simplicity that their assignment is random.
2.  Update - Determine the new centroid by taking the mean of each cluster.

We then repeat these two steps for a set number of iterations or until convergence (where the centroids have a fixed position and no updates take place).

K-Means makes a number of assumptions:
1.  We know $K$ in advance
2.  All data points belong to a cluster - i.e., there is no 'noise'

Let's see an example.

## Example
### The Wine Dataset

In [ ]:
%pylab inline
import pandas as pd

from sklearn.datasets import load_wine

rcParams['figure.figsize'] = (20,7)


wine = load_wine()

In [ ]:
# Create dataframe with features
df = pd.DataFrame(wine['data'], columns=wine['feature_names'])

# Unsupervised - no target!

print(df.head())

print(wine['DESCR'])

As before, we'll dispense with exploratory data analysis as we know the data is complete and clean.  

### Setting Up the Experiment

Notice that in this case, as the dataset was originally designed for classification, we know the number of clusters that should be in the final result.  In real-world problems, we will not know this.  Let's see if our analysis can lead us to this number.

In [ ]:
print("Number of classes of wine: ", len(np.unique(wine['target'])))

As we've seen before, a good way to get some intuition for a dataset is to plot it.  Here we have 13 features, so it's going to be a bit difficult to plot them all together.  Instead, let's use a technique for *dimensionality reduction* - we convert our 13-dimensional space into a 2 (or 3) dimensional space that attempts to maintain the distances between the individual data points.  Here we'll use a technique called *T-SNE*.

In [ ]:
from sklearn.manifold import TSNE

# Reduce into 2 dimensions for plotting.  Use a random_state constant for repeatability.
tsne = TSNE(n_components=2, random_state=654456)
data_for_plotting = tsne.fit_transform(df)

# The data_for_plotting variable has a dimension of 2 - let's put one on the x axis and the other on the y axis
scatter(data_for_plotting[:,0], data_for_plotting[:,1])

OK, so that wasn't a big help - nothing obvious here.  But let's look at the data again, particularly at the mean and standard deviation:

In [ ]:
df.describe().loc[['mean','std']]

Notice that we a have a 2 order-of-magnitude difference between the smallest and largest means!  This means that the features with the largest values will account for most of the variability in the data.  This is not good for us.  Let's see if we can *scale* the data to a common value.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale each feature to a mean of 0 and standard deviation of 1
scaled_data = StandardScaler().fit_transform(df)

# Use the scaled data with the same column names
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

# Show mean and std again
scaled_df.describe().loc[['mean','std']]

How does this affect our plot?

In [ ]:
tsne = TSNE(n_components=2, random_state=654456)

# Apply T-SNE to the data frame with the scaled data
data_for_plotting = tsne.fit_transform(scaled_df )

# Using the target values for color is a bit of cheating...
scatter(data_for_plotting[:,0], data_for_plotting[:,1], c=wine['target'])

Much better!  We also cheated and used the data to show us the cluster.  Interestingly, we see that:
1.  The clusters - in general - can be bounded by ellipses.  If this is not the case, K-means will not perform well and we need a different algorithm.
2.  Some points seem to 'live' inside a different cluster.  K-means will most likely miss-label on these points.

### Running the Algorithm

Let's now try running the algorithm, for various values of K:

In [ ]:
from sklearn.cluster import KMeans

fig, axs = subplots(2,3)

for ax, k in zip(np.ravel(axs), range(2,8)):
    km = KMeans(n_clusters=k, random_state=123)
    km.fit(scaled_df)
    ax.scatter(data_for_plotting[:,0], data_for_plotting[:,1], c = km.labels_)
    ax.set_title('Cluster assignment, $k$ = {}'.format(k))


A visual inspection shows us that 3 clusters seems to be the best option - and note that indeed we have some mislabeled points (of course, in a real-world problem we wouldn't know that - but we still would know there are 3 clusters).  But what's to say that 8 clusters aren't better?  Or 20?  Or 100?  How can we quantify and measure the relative values of a number of clusters?

### Silhouette Score & The Elbow Method

In order to find the best assignment, we need to assign a score to each assignment.  For that we use the *Silhouette Score*.  In essence, it measures how similiar a point is to the other points in its cluster vs. points in the nearest different cluster.  The silhouette score ranges from -1 to 1, with 1 being the highest score possible.

Let's compute the score for a number of clusters:

In [ ]:
from sklearn.metrics import silhouette_score


scores = []
for k in range(2,30):
    km = KMeans(n_clusters=k, random_state=123)
    km.fit(scaled_df)

    scores.append(silhouette_score(scaled_df, km.labels_))
    
plot(range(2,30), scores)
vlines(argmax(scores)+2, 0, max(scores), color='red', linestyles='--')
grid(b='both')
xlabel("# of clusters")
ylabel("Silhouette Score")
print("Best silhouette value is at {} clusters".format(argmax(scores)+2))

The shilouette score is very useful, but let's also see a different method.  The KMeans object itself has a property named `inertia_` - the trailing underscore means that this value is set only *after* the `fit` method has been run.  This property measures the *sum of the squared distances* from each data point to the centroid (i.e., cluster center) to which it is assigned.  Obviously, the lower score the better.  But there's a catch - let's see.

In [ ]:
inertias = []
for k in range(2,30):
    km = KMeans(n_clusters=k, random_state=123)
    km.fit(scaled_df)

    inertias.append(km.inertia_)
    
plot(range(2,30), inertias)
grid(axis='both')
xlabel('# of clusters')
ylabel('Sum of Squares to centroid')
vlines(3, 0, inertias[3-2], color='red', linestyles='--')

We see that the sum-of-squares value keep decreasing.  This is not surprising - the more clusters we add the less error there is.  In fact, if each point is in a separate cluster the error is zero.  But that's not useful for us.  So how to interpret this plot?

We use the *Elbow Method*.  Notice that the line we drew has 2 'behaviors' - one steep line from 2 to 3 clusters, and then a more shallow line that goes from 3 clusters on.  At 3 clusters we have an 'elbow' - the transition from the steep to the shallow behavior.  The value at this elbow is often the best number of clusters.  Since this coincides nicely with the silhouette score we can be reasonably confident that 3 is indeed the correct number of clusters.

## Summary

In this notebook we took a look at the *K-Means* algorithm for clustering.  We saw that it requires a number $k$ and it *partitions* the data into that many distinct subsets.  We further saw some ways to measure what the best number of clusters is for the data.

K-Means is often used for clustering problems as it is computationally efficient and has many good implementations.  

## Exercises

1.  Use the `Perfume` dataset from the Machine Learning Repository at the University of California at Irvine (https://archive.ics.uci.edu/ml/datasets/Perfume+Data) and repeat the analysis above.  How many clusters did you find?
